In [ ]:
# Have Decided to use 50bp for the window size
# This is the process to convert txt files into bedgraph files 
import pandas as pd
import glob
import os

def batch_generate_50bp_level(input_dir=".", bin_size=50, min_cov=10):
    # 1. 查找目录下所有 .txt 文件
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    
    if not txt_files:
        print("未找到任何 .txt 文件，请检查路径。")
        return

    for txt_file in txt_files:
        output_bg = txt_file.replace(".txt", f".bin{bin_size}.bedgraph")
        print(f"--- 正在处理: {os.path.basename(txt_file)} ---")
        
        # 2. 读取数据 (针对你的 CX report 格式)
        # 0:chr, 1:pos, 3:met, 4:unmet
        try:
            # 使用 chunk 模式读取以防内存崩溃
            chunks = pd.read_csv(txt_file, sep='\t', header=None, 
                                 usecols=[0, 1, 3, 4], 
                                 names=['chr', 'pos', 'met', 'unmet'],
                                 chunksize=2000000)
            
            # 创建一个列表来存储每个 chunk 聚合后的结果
            aggregated_data = []
            
            for chunk in chunks:
                # 计算 50bp bin
                chunk['bin_start'] = (chunk['pos'] // bin_size) * bin_size
                # 组内求和
                binned_chunk = chunk.groupby(['chr', 'bin_start']).agg({'met': 'sum', 'unmet': 'sum'}).reset_index()
                aggregated_data.append(binned_chunk)
            
            # 3. 二次聚合：确保跨 chunk 的同一个 bin 被合并
            full_binned = pd.concat(aggregated_data).groupby(['chr', 'bin_start']).agg({'met': 'sum', 'unmet': 'sum'}).reset_index()
            
            # 4. 计算 Level 并过滤
            full_binned['total'] = full_binned['met'] + full_binned['unmet']
            valid = full_binned[full_binned['total'] >= min_cov].copy()
            valid['level'] = (valid['met'] / valid['total']).round(4) # 保留4位小数
            valid['bin_end'] = valid['bin_start'] + bin_size
            
            # 5. 按照 BedGraph 标准格式排序并保存
            # 标准格式：chr, start, end, value
            valid = valid.sort_values(['chr', 'bin_start'])
            valid[['chr', 'bin_start', 'bin_end', 'level']].to_csv(
                output_bg, sep='\t', index=False, header=False
            )
            
            print(f"成功导出: {output_bg}")
            
        except Exception as e:
            print(f"处理 {txt_file} 时出错: {e}")

# 在 Jupyter 中直接运行：
batch_generate_50bp_level()

In [ ]:
# This is the file to convert bedgraph into bigwig files
import pyBigWig
import pandas as pd
import glob
import os

def batch_convert_to_bigwig(chrom_sizes_file, input_pattern="*.bin50.bedgraph"):
    # 1. 加载染色体大小并记录顺序 (BigWig 对顺序有严格要求)
    print("正在加载染色体大小...")
    chrom_order = []
    chrom_sizes_dict = {}
    with open(chrom_sizes_file, 'r') as f:
        for line in f:
            if line.strip():
                parts = line.split()
                chrom_order.append(parts[0]) 
                chrom_sizes_dict[parts[0]] = int(parts[1])
    
    # 2. 查找匹配的文件
    files = glob.glob(input_pattern)
    if not files:
        print(f"未找到匹配 '{input_pattern}' 的文件，请检查路径。")
        return

    print(f"找到 {len(files)} 个文件准备转换...")

    for bedgraph_file in files:
        # 设定输出文件名 (把 .bedgraph 替换为 .bw)
        output_bw = bedgraph_file.replace(".bedgraph", ".bw")
        
        print(f"\n--- 正在处理: {bedgraph_file} ---")
        
        try:
            # 创建 BigWig
            bw = pyBigWig.open(output_bw, "w")
            bw.addHeader([(chrom, chrom_sizes_dict[chrom]) for chrom in chrom_order])
            
            # 读取数据
            df = pd.read_csv(bedgraph_file, sep='\t', header=None)
            df.columns = ['chrom', 'start', 'end', 'value']
            
            # 按照 chrom.sizes 的顺序写入 (BigWig 必须按 header 顺序写入)
            for chrom in chrom_order:
                group = df[df['chrom'] == chrom]
                if not group.empty:
                    group = group.sort_values('start')
                    # 写入 BigWig
                    bw.addEntries(
                        group['chrom'].tolist(),
                        group['start'].tolist(),
                        ends=group['end'].tolist(),
                        values=group['value'].tolist()
                    )
            
            bw.close()
            print(f"✅ 转换成功: {output_bw}")
            
        except Exception as e:
            print(f"❌ 处理 {bedgraph_file} 时出错: {e}")

# --- 执行设置 ---
# 确保你的 hg38.chrom.sizes 在当前目录下
my_sizes = 'hg38.chrom.sizes' 

# 执行批量转换
batch_convert_to_bigwig(my_sizes)

In [ ]:
%%bash
# The process that deeptools can compare two bw files based on set window size
bigwigCompare -b1 675722-0427605773.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport.bin50.bw -b2 675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport.bin50.bw \
    -o gCGG_vs_gScr_modc_diff.bw \
    --operation log2 \
    --pseudocount 1.0 \
    --binSize 50 \
    --p 4

In [ ]:
#The way we unzip CGG-CCG gz file 
import gzip
import shutil

input_file = 'CGG-CCG_repeatMasker.hg38.sorted.bed.gz'
output_file = 'CGG-CCG_repeatMasker.hg38.sorted.bed'

print(f"📂 正在解压 {input_file}...")
with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print(f"✅ 解压完成！新文件位于: {output_file}")

In [ ]:
%%bash
# The way we try to plot heatmap with CGG-CCG at the center  
computeMatrix reference-point --referencePoint center \
  -S gCGG_vs_gScr_mc_diff.bw \
  -R CGG-CCG_repeatMasker.hg38.sorted.bed \
  -b 1000 -a 1000 \
  -bs 50 \
  -o gCGG_vs_gScr_5mc_50bp.gz \
  --missingDataAsZero

# 绘制没有聚类的组合图（Profile + Heatmap）
plotHeatmap -m gCGG_vs_gScr_5mc_50bp.gz \
  -out gCGG_vs_gScr_5mc_50bp.pdf \
  --colorMap RdPu \
  --zMin 0 --zMax 0.3 \
  --regionsLabel "Total CGG Repeats" \
  --refPointLabel "CGG" \
  --heatmapHeight 25 \
  --plotTitle "Global 6bp-seq Signal at CGG Sites (50bp)" \
  --whatToShow "plot, heatmap and colorbar" \
  --averageType mean

In [ ]:
%%bash
# or with K-means 
plotHeatmap -m gCGG_vs_gScr_5mc_50bp.gz \
  -out gCGG_vs_gScr_5mc_50bp_kmeans5.pdf \     # we can change the k values
  --kmeans 5 \
  --colorMap RdPu \
  --zMin 0 --zMax 0.3 \
  --regionsLabel "Cluster 1" "Cluster 2" "Cluster 3" "Cluster 4" "Cluster 5" \
  --refPointLabel "CGG" \
  --heatmapHeight 25 \
  --plotTitle "Unbiased 6bp-seq Analysis (50bp bins)" \
  --whatToShow "plot, heatmap and colorbar" \
  --averageType mean